# Notebook Setup

In [ ]:
import pandas as pd
import seaborn as sb

from IPython.display import display

In [ ]:
MEASURES_FILENAME = "./measures.csv"
CLOCK_ADJUST_RECTYPE = "CLOCK_ADJUSTED"
MEASURE_RECTYPE = "REGULAR"

COSC_LIMITS_SEC_DAY = (-4, 6)

GRAPH_STYLE = "whitegrid"

In [ ]:
sb.set_theme("notebook", GRAPH_STYLE)
sb.set_style()

pd.options.display.max_rows = 6

# Read Data

In [ ]:
# Read the csv
data = pd.read_csv(MEASURES_FILENAME)

# Put data back in chronological order
data = data.iloc[::-1].reset_index(drop=True)

In [ ]:
# Rename columns to make coding comfy
data = data.rename(columns={
    "Type of record":"rectype",
    "Offset (seconds)":"offset"
})

In [ ]:
data = data.astype({"rectype":"category"})

## Extract machine readable timestamps (utc)

In [ ]:
data["timestamp"] = pd.to_datetime(data["Timestamp (Epoch Time)"], unit="ms")

## Create Unique Chain Numbers

In [ ]:
data["seriesID"] = data.rectype.eq(CLOCK_ADJUST_RECTYPE).cumsum().shift(1, fill_value=0)

## Separate Measures

In [ ]:
measures = (
    data
    .query("rectype == @MEASURE_RECTYPE")
    .copy()
    .drop(["rectype", "Series Accuracy (Seconds/day)"], axis="columns")
)
measures

## Separate Chain Infos

In [ ]:
seriesInfo = (
    data
    .query("rectype == @CLOCK_ADJUST_RECTYPE")
    .set_index("seriesID")
    .loc[:, ["timestamp", "Timestamp (Human Readable)", "Series Accuracy (Seconds/day)"]]
    .rename({
        "timestamp":"seriesEnd",
        "Timestamp (Human Readable)":"seriesEnd (Human Readable)",
        "Series Accuracy (Seconds/day)":"appDriftRate"
    }, axis="columns")
)

seriesInfo

## Relative Time Within Series

In [ ]:
seriesStarts = measures.groupby("seriesID").timestamp.min()

measures["adjustmentAge"] = measures.timestamp - measures.seriesID.map(seriesStarts)
measures["adjustmentDays"] = measures.adjustmentAge.dt.total_seconds() / 3600 / 24

seriesInfo = seriesInfo.merge(seriesStarts.rename("seriesStart"), "outer", left_index=True, right_index=True)

## Spot Rates

In [ ]:
measures["spotRate"] = measures.offset / measures.adjustmentDays

## Quick Look at the Data

In [ ]:
measures

In [ ]:
seriesInfo

# Plot Offsets

In [ ]:
f = sb.relplot(data=measures, x="timestamp", y="offset", hue="seriesID", kind="line", aspect=2)
f.figure.autofmt_xdate()

# Plot Superposed Adjustment Cycles

In [ ]:
sb.set_style("ticks")
f = sb.lmplot(data=measures, x="adjustmentDays", y="offset", hue="seriesID", height=10)

maxDays = measures.adjustmentDays.max()
minOffset = measures.offset.min()

limitsX = [0, maxDays]
limitYMin = [0, maxDays * COSC_LIMITS_SEC_DAY[0]]
limitYMax = [0, maxDays * COSC_LIMITS_SEC_DAY[1]]

f.ax.fill_between(limitsX, limitYMin, limitYMax, color="green", alpha=0.1, zorder=0)

for i in range(-30,31):
    f.ax.axline((0,0), (1/100, i/100), c="lightgrey", lw=0.5, ls="--")

f.ax.axhline(0, c="black", lw=2, ls="--")

# Get Drift Rates

In [ ]:
from scipy.stats import linregress

def chainStats(df):
    
    linreg = linregress(df.adjustmentDays, df.offset)
    
    initalSampleCount = df.index.size
    
    df = df.query("abs(offset) > 0")
    
    return pd.Series(
        {
            "initialSamples":initalSampleCount,
            "nonZeroSamples":df.index.size,
            "finalOffset":df.offset.iat[-1],
            "observationDays":df.adjustmentDays.iat[-1],
            "median":df.spotRate.median(),
            "mean":df.spotRate.mean(),
            "stddev":df.spotRate.std(),
            "final":df.spotRate.iat[-1],
            "avgFinal3":df.spotRate.tail(3).mean(),
            "slope":linreg.slope, "R":linreg.rvalue,
            
        })

stats = measures.groupby("seriesID").apply(chainStats)
stats = stats.join(seriesInfo.appDriftRate).join(seriesInfo.seriesStart.dt.date)
(
    stats
    .style
    .format("{:.0f}", pd.IndexSlice[:, :"observationDays"])
    .format("{:.2f}", pd.IndexSlice[:, "observationDays":"appDriftRate"], na_rep="...")
)